In [42]:
import pprint
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from json_shot_scraper import flatten_shot, flatten_goal, flatten_complete_pass, flatten_incomplete_pass, flatten_corner
from  player_scraper import flatten_player, flatten_sub
from dataframe_cleaner import (pass_to_shot, corner_to_shot, transpose_coordinates, coord_to_yards, 
                               shot_distance_angle, dummy_columns, drop_own_goals, goal_dummy)
from model_prep import create_hypothetical_df
from transfer_market_scraper import flatten_player_info

In [4]:
pd.set_option('display.max_columns', 50)

In [63]:
from html_scraper import db
from mongo_to_db import game_to_cleaned_df, create_frame, create_master_df, create_master_player_min_df
import string

# players to match up data with

In [13]:
games = db.games.find()
players_minutes_df = create_master_player_min_df(games)

In [244]:
players_minutes_df['squad_number'].unique()

array([22., 12., 24.,  6., 29., 34.,  4., 15., 16., 30., 28.,  8., 23.,
       10., 11., 26.,  9., 33., nan, 13., 20., 21.,  5., 18., 31.,  7.,
       19.,  3., 14., 25.,  2., 27., 41., 40., 37., 32.,  1., 17., 39.,
       38., 87., 35., 36., 42., 43.])

In [245]:
players_minutes_df.head()

,game_id,name,player_id,position_id,squad_number,substitute,team_id,minutes_played
0,448548,Alexander Domínguez,19132.0,1,22.0,False,20,90.0
1,448548,Lucas Hoyos,27987.0,1,12.0,True,20,0.0
2,448548,Gastón Díaz,30896.0,2,24.0,False,20,90.0
3,448548,Joaquín Laso,94225.0,2,6.0,False,20,90.0
4,448548,Luis Abram,99779.0,2,29.0,False,20,90.0


In [247]:
names = players_minutes_df['name'].unique()

In [291]:
afa_team_dict = {20: 'VEL', 13: 'NOB', 136: 'TIG', 19: 'SLO', 8: 'GIM', 2: 'ARG',
 137: 'UNI', 122: 'ALD', 869: 'PA', 6: 'COL', 124: 'BEL', 134: 'SMS',
 5: 'BOC', 135: 'TAL', 132: 'GOD', 7: 'EST', 12: 'LAN', 129: 'DEF',
 18: 'ROS', 4: 'BAN', 100: 'HUR', 17: 'RIV', 815: 'ATT', 16: 'RAC', 10: 'IND',
 490: 'SMT'}

In [292]:
len(afa_team_dict)

26

In [262]:
team_dict = [{'team_id': '20', 'team_intitials': 'VEL', 'short_name': 'Vélez'},
 {'team_id': '13', 'team_intitials': 'NOB', 'short_name': 'Newell`s'},
 {'team_id': '136', 'team_intitials': 'TIG', 'short_name': 'Tigre'},
 {'team_id': '19', 'team_intitials': 'SLO', 'short_name': 'San Lorenzo'},
 {'team_id': '8', 'team_intitials': 'GIM', 'short_name': 'Gimnasia'},
 {'team_id': '2', 'team_intitials': 'ARG', 'short_name': 'Argentinos'},
 {'team_id': '137', 'team_intitials': 'UNI', 'short_name': 'Unión'},
 {'team_id': '122', 'team_intitials': 'ALD', 'short_name': 'Aldosivi'},
 {'team_id': '869', 'team_intitials': 'PA', 'short_name': 'Patronato'},
 {'team_id': '6', 'team_intitials': 'COL', 'short_name': 'Colón'},
 {'team_id': '124', 'team_intitials': 'BEL', 'short_name': 'Belgrano'},
 {'team_id': '134', 'team_intitials': 'SMS', 'short_name': 'S. Martín SJ'},
 {'team_id': '5', 'team_intitials': 'BOC', 'short_name': 'Boca'},
 {'team_id': '135', 'team_intitials': 'TAL', 'short_name': 'Talleres'},
 {'team_id': '132', 'team_intitials': 'GOD', 'short_name': 'Godoy Cruz'},
 {'team_id': '7', 'team_intitials': 'EST', 'short_name': 'Estudiantes'},
 {'team_id': '12', 'team_intitials': 'LAN', 'short_name': 'Lanús'},
 {'team_id': '129', 'team_intitials': 'DEF', 'short_name': 'Defensa'},
 {'team_id': '18', 'team_intitials': 'ROS', 'short_name': 'R. Central'},
 {'team_id': '4', 'team_intitials': 'BAN', 'short_name': 'Banfield'},
 {'team_id': '100', 'team_intitials': 'HUR', 'short_name': 'Huracán'},
 {'team_id': '17', 'team_intitials': 'RIV', 'short_name': 'River'},
 {'team_id': '815', 'team_intitials': 'ATT', 'short_name': 'Atl. Tucumán'},
 {'team_id': '16', 'team_intitials': 'RAC', 'short_name': 'Racing'},
 {'team_id': '10', 'team_intitials': 'IND', 'short_name': 'Independiente'},
 {'team_id': '490', 'team_intitials': 'SMT', 'short_name': 'S. Martín T'}]

In [293]:
players_minutes_df['club_brev'] = players_minutes_df['team_id'].map(afa_team_dict)

In [295]:
players_minutes_df.head()

,game_id,name,player_id,position_id,squad_number,substitute,team_id,minutes_played,club_brev
0,448548,Alexander Domínguez,19132.0,1,22.0,False,20,90.0,VEL
1,448548,Lucas Hoyos,27987.0,1,12.0,True,20,0.0,VEL
2,448548,Gastón Díaz,30896.0,2,24.0,False,20,90.0,VEL
3,448548,Joaquín Laso,94225.0,2,6.0,False,20,90.0,VEL
4,448548,Luis Abram,99779.0,2,29.0,False,20,90.0,VEL


# Transfer Market Data

In [237]:
players = db.players.find()

In [238]:
master_list = []
for player in players:
    master_list.append(player)

In [240]:
tm_player = pd.DataFrame(master_list)


In [241]:
tm_player.head()

,_id,birthday,club,foot,height,player,squad_num,transfer_value(sterlings)
0,5c0588e63d54c00a19927577,"Jan 26, 1991 (27)",Squad Club Atlético Boca Juniors,right,"1,94 m",Esteban Andrada,31,£4.50m
1,5c0588e73d54c00a19927578,"Mar 17, 1987 (31)",Squad Club Atlético Boca Juniors,right,"1,92 m",Carlos Lampe,28,£495k
2,5c0588e73d54c00a19927579,"Sep 27, 1993 (25)",Squad Club Atlético Boca Juniors,right,"1,85 m",Lisandro Magallán,6,£4.05m
3,5c0588e73d54c00a1992757a,"May 12, 1985 (33)",Squad Club Atlético Boca Juniors,right,"1,84 m",Paolo Goltz,2,£1.62m
4,5c0588e73d54c00a1992757b,"Feb 22, 1991 (27)",Squad Club Atlético Boca Juniors,left,"1,73 m",Frank Fabra,18,£5.40m


### Turn squad numbers into floats...

In [243]:
tm_player['squad_num'].unique()

array(['31', '28', '6', '2', '18', '20', '29', '16', '15', '8', '30', '7',
       '22', '19', '17', '1', '25', '24', '26', '3', '14', '4', '40', '5',
       '39', '10', '11', '32', '9', '13', '-', '23', '21', '27', '41',
       '34', '37', '12', '36', '38', '33', '35', '87'], dtype=object)

In [300]:
tm_player.loc[tm_player.squad_num == '-', 'squad_num'] = 99

In [278]:
tm_player['squad_num'].unique()

array(['31', '28', '6', '2', '18', '20', '29', '16', '15', '8', '30', '7',
       '22', '19', '17', '1', '25', '24', '26', '3', '14', '4', '40', '5',
       '39', '10', '11', '32', '9', '13', '', '23', '21', '27', '41',
       '34', '37', '12', '36', '38', '33', '35', '87'], dtype=object)

In [299]:
tm_player = tm_player.astype({'squad_num': int})

ValueError: invalid literal for int() with base 10: ''

### Add Column of Club_Abbrev

In [265]:
tm_player['club'].unique()

array(['Squad Club Atlético Boca Juniors',
       'Squad Club Atlético Independiente',
       'Squad Club Atlético River Plate', 'Squad Racing Club',
       'Squad Club Atlético San Lorenzo de Almagro',
       'Squad Club Atlético Lanús', 'Squad Club Atlético Vélez Sarsfield',
       'Squad Club Deportivo Godoy Cruz Antonio Tomba',
       'Squad Club Atlético Colón',
       'Squad Club Social y Deportivo Defensa y Justicia',
       'Squad Club Atlético Rosario Central',
       'Squad Club Atlético Talleres',
       'Squad Club Atlético Unión (Santa Fe)',
       'Squad Club Estudiantes de La Plata',
       'Squad Club Atlético Huracán',
       'Squad Asociación Atlética Argentinos Juniors',
       'Squad Club Atlético Banfield', 'Squad Club Atlético Belgrano',
       'Squad Club Atlético Tigre',
       "Squad Club Atlético Newell's Old Boys",
       'Squad Club Atlético Tucumán',
       'Squad Club de Gimnasia y Esgrima La Plata',
       'Squad Club Atlético Aldosivi',
       'Squad Clu

In [268]:
tm_club_brev = {'Squad Club Atlético Boca Juniors': 'BOC' , 'Squad Club Atlético Independiente': 'IND', 'Squad Club Atlético River Plate': 'RIV', 'Squad Racing Club': 'RAC',
       'Squad Club Atlético San Lorenzo de Almagro': 'SLO', 'Squad Club Atlético Lanús': 'LAN', 'Squad Club Atlético Vélez Sarsfield': 'VEL',
       'Squad Club Deportivo Godoy Cruz Antonio Tomba': 'GOD', 'Squad Club Atlético Colón': 'COL', 'Squad Club Social y Deportivo Defensa y Justicia': 'DEF',
       'Squad Club Atlético Rosario Central': 'ROS', 'Squad Club Atlético Talleres': 'TAL', 'Squad Club Atlético Unión (Santa Fe)': 'UNI',
       'Squad Club Estudiantes de La Plata': 'EST', 'Squad Club Atlético Huracán': 'HUR', 'Squad Asociación Atlética Argentinos Juniors': 'ARG',
       'Squad Club Atlético Banfield': 'BAN', 'Squad Club Atlético Belgrano': 'BEL', 'Squad Club Atlético Tigre': 'TIG', "Squad Club Atlético Newell's Old Boys": 'NOB',
       'Squad Club Atlético Tucumán': 'ATT', 'Squad Club de Gimnasia y Esgrima La Plata': 'GIM', 'Squad Club Atlético Aldosivi': 'ALD',
       'Squad Club Atlético San Martín (SJ)': 'SMS', 'Squad Club Atlético San Martín (Tucumán)': 'SMT', 'Squad Club Atlético Patronato de la Juventud Católica': 'PA'}

In [269]:
tm_player.head()

,_id,birthday,club,foot,height,player,squad_num,transfer_value(sterlings)
0,5c0588e63d54c00a19927577,"Jan 26, 1991 (27)",Squad Club Atlético Boca Juniors,right,"1,94 m",Esteban Andrada,31,£4.50m
1,5c0588e73d54c00a19927578,"Mar 17, 1987 (31)",Squad Club Atlético Boca Juniors,right,"1,92 m",Carlos Lampe,28,£495k
2,5c0588e73d54c00a19927579,"Sep 27, 1993 (25)",Squad Club Atlético Boca Juniors,right,"1,85 m",Lisandro Magallán,6,£4.05m
3,5c0588e73d54c00a1992757a,"May 12, 1985 (33)",Squad Club Atlético Boca Juniors,right,"1,84 m",Paolo Goltz,2,£1.62m
4,5c0588e73d54c00a1992757b,"Feb 22, 1991 (27)",Squad Club Atlético Boca Juniors,left,"1,73 m",Frank Fabra,18,£5.40m


In [274]:
tm_player['club_brev'] = tm_player['club'].map(tm_club_brev)

In [276]:
tm_player.head()

,_id,birthday,club,foot,height,player,squad_num,transfer_value(sterlings),club_brev
0,5c0588e63d54c00a19927577,"Jan 26, 1991 (27)",Squad Club Atlético Boca Juniors,right,"1,94 m",Esteban Andrada,31,£4.50m,BOC
1,5c0588e73d54c00a19927578,"Mar 17, 1987 (31)",Squad Club Atlético Boca Juniors,right,"1,92 m",Carlos Lampe,28,£495k,BOC
2,5c0588e73d54c00a19927579,"Sep 27, 1993 (25)",Squad Club Atlético Boca Juniors,right,"1,85 m",Lisandro Magallán,6,£4.05m,BOC
3,5c0588e73d54c00a1992757a,"May 12, 1985 (33)",Squad Club Atlético Boca Juniors,right,"1,84 m",Paolo Goltz,2,£1.62m,BOC
4,5c0588e73d54c00a1992757b,"Feb 22, 1991 (27)",Squad Club Atlético Boca Juniors,left,"1,73 m",Frank Fabra,18,£5.40m,BOC
